# CrewAI Agents with Llama Stack and RAG

This notebook demonstrates **CrewAI** integration with **Llama Stack** to build a Retrieval-Augmented Generation (RAG) system.

## Overview

- **Llama Stack**: Provides the infrastructure for running LLMs and vector store
- **CrewAI**: Offers a framework for orchestrating agents and tasks
- **Integration**: Leverages Llama Stack's OpenAI-compatible API with CrewAI

## Configuration

This notebook uses environment variables from `.env` file in the project root.
Create your own `.env` file based on `.env.example`.

## Architecture

```
┌─────────────────────────────────────────────┐
│         CrewAI Agent                        │
│         (Agent + Task + Crew)               │
└────────┬────────────────────────────────────┘
         │                    
         │ Model Calls        
         ▼                    
┌─────────────────┐   ┌──────────────────────┐
│  Llama Stack    │   │  RAG Tool            │
│  (OpenAI API)   │   │  (CrewAI)            │
│  - vLLM Engine  │──▶│  - Vector Search     │
│  - Inference    │   │  - Document Retrieval│
└─────────────────┘   └──────┬───────────────┘
                             │
                      Tool Execution
                             ▼
                   ┌──────────────────┐
                   │ Llama Stack      │
                   │ Vector Store API │
                   └──────────────────┘
```

## Install Required Packages

Install CrewAI and dependencies:

In [ ]:
%pip install -q --upgrade \
  "crewai>=0.76.0" \
  "crewai-tools>=0.33.0" \
  "chromadb>=0.5.5" \
  "pydantic<3" \
  "pysqlite3-binary>=0.5.3.post3" \
  "ipywidgets==8.1.8"

In [ ]:
import sys
import importlib

# Force Python to use the modern SQLite provided by pysqlite3-binary
pysqlite3 = importlib.import_module("pysqlite3")
sys.modules["sqlite3"] = pysqlite3
sys.modules["sqlite"] = pysqlite3  # some libs import 'sqlite' directly

# (Optional) sanity check
import sqlite3
print("SQLite version:", sqlite3.sqlite_version)  # should be >= 3.35.0

## Import Dependencies

In [ ]:
# Core imports
import os
import json
from pathlib import Path
from pprint import pprint
from typing import Any, List, Optional, Type
from dotenv import load_dotenv, find_dotenv
from io import BytesIO

# Llama Stack client
from llama_stack_client import LlamaStackClient

# CrewAI imports
from crewai import Agent, Crew, Task
from crewai.llm import LLM
from crewai.tools import BaseTool
from pydantic import BaseModel, Field

# --- Load environment variables ---
# Automatically detect the nearest .env (walks up from current directory)
env_path = find_dotenv(usecwd=True)
if env_path:
    load_dotenv(env_path)
    print(f"📁 Loading environment from: {env_path}")
    print("✅ .env file FOUND and loaded")
else:
    default_path = Path.cwd() / ".env"
    print(f"📁 No .env found via find_dotenv — checked: {default_path}")
    print("⚠️  .env file NOT FOUND")

# --- Verify Python interpreter / kernel ---
print(f"\n🐍 Python: {sys.executable}")

# Detect if running inside a virtual environment
in_venv = (
    hasattr(sys, "real_prefix") or
    (getattr(sys, "base_prefix", sys.prefix) != sys.prefix) or
    "VIRTUAL_ENV" in os.environ or
    "CONDA_PREFIX" in os.environ
)

if in_venv:
    print("✅ Using virtual environment - CORRECT!")
else:
    print("⚠️  Using global Python - Consider switching kernel!")
    print("   Click 'Select Kernel' → Choose 'Python (byo-agentic-framework)')")

## Configure Llama Stack Connection

Connect to Llama Stack's OpenAI-compatible endpoint:

In [ ]:
# === Llama Stack Configuration ===
# Load from environment variables
LLAMA_STACK_BASE_URL = os.getenv("LLAMA_STACK_BASE_URL")
LLAMA_STACK_OPENAI_ENDPOINT = os.getenv("LLAMA_STACK_OPENAI_ENDPOINT")
INFERENCE_MODEL = os.getenv("INFERENCE_MODEL")
API_KEY = os.getenv("API_KEY", "fake")

print("🌐 Llama Stack Configuration:")
print(f"   Base URL: {LLAMA_STACK_BASE_URL}")
print(f"   OpenAI Endpoint: {LLAMA_STACK_OPENAI_ENDPOINT}")
print(f"   Model: {INFERENCE_MODEL}")

# Initialize Llama Stack Client
from llama_stack_client import LlamaStackClient

client = LlamaStackClient(
    base_url=LLAMA_STACK_BASE_URL,
)

print("✅ LlamaStack client initialized")

# Create CrewAI LLM instance pointing to Llama Stack
from crewai.llm import LLM

llm = LLM(
    model=f"openai/{INFERENCE_MODEL}",  # OpenAI-compatible model format
    base_url=LLAMA_STACK_OPENAI_ENDPOINT,
    api_key=API_KEY,
    temperature=0.0,
)

print("\\n🧪 Testing connectivity...")
try:
    response = llm.call([
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Say 'Connection successful' if you can read this."}
    ])
    print(f"📥 LLM Response: {response}")
    print("✅ Llama Stack connection successful!")
except Exception as e:
    print(f"❌ Connection failed: {e}")
    sys.exit(1)

## Vector Store Setup

### Create a Vector Store with Sample Documents

Create a vector store using Llama Stack's OpenAI-compatible vector stores API:

- **Vector Store**: OpenAI-compatible vector store for document storage
- **File Upload**: Automatic chunking and embedding of uploaded files  
- **Embedding Model**: Sentence Transformers model for text embeddings
- **Dimensions**: 384-dimensional embeddings

In [ ]:
print("📚 Setting up vector store with Acme documentation...\n")

# Sample documents for RAG
docs = [
    ("Acme ships globally in 3-5 business days.", {"title": "Shipping Policy"}),
    ("Returns are accepted within 30 days of purchase.", {"title": "Returns Policy"}),
    ("Support is available 24/7 via chat and email.", {"title": "Support"}),
]

# Upload files using Llama Stack client
file_ids = []
print("📤 Uploading documents to Llama Stack...")
for content, metadata in docs:
    # Llama Stack's file API expects a file object (not string), so we create an in-memory file.
    with BytesIO(content.encode()) as file_buffer:
        # Set a filename for the in-memory file
        file_buffer.name = f"{metadata['title'].replace(' ', '_').lower()}.txt"
        # Upload to Llama Stack
        create_file_response = client.files.create(
            file=file_buffer,
            purpose="assistants"
        )
        print(f"   ✅ Uploaded: {metadata['title']} (ID: {create_file_response.id})")
        # Store file ID for vector store creation to later use
        file_ids.append(create_file_response.id)

# Create vector store with uploaded files
print("\n🔧 Creating vector store...")
vector_store = client.vector_stores.create(
    name="acme_docs",
    file_ids=file_ids,  # Use uploaded file IDs
    #embedding_model="sentence-transformers/all-MiniLM-L6-v2",
    #embedding_dimension=384,
    #provider_id="milvus" # Use Milvus as the vector store provider
)

print(f"✅ Vector store created successfully!")
print(f"   ID: {vector_store.id}")
print(f"   Name: {vector_store.name}")

### Test Knowledge Base Search

Test our simple keyword-based search functionality:

In [ ]:
print("🔍 Testing vector search...\n")

# Perform semantic search on the vector store
search_response = client.vector_stores.search(
    vector_store_id=vector_store.id,
    query="How long does shipping take?",
    max_num_results=2
)

# Display the most relevant documents found
print("📊 Search Results:")
for result in search_response.data:
    content = result.content[0].text
    print(f"   - {content}")

print("\n✅ Vector search working correctly!")

## Create CrewAI Custom RAG Tool

Define a custom CrewAI tool to query the knowledge base:

- **Input Schema**: Defines the user query and optional parameters like `top_k`
- **Tool Logic**: Performs keyword-based search and returns formatted results

In [ ]:
# ---------- 1. Input schema ----------
class VectorStoreRAGToolInput(BaseModel):
    """Input schema for LlamaStackVectorStoreRAGTool."""
    query: str = Field(..., description="The user query for RAG search")
    vector_store_id: str = Field(...,
        description="ID of the vector store to search inside the Llama-Stack server",
    )
    top_k: Optional[int] = Field(
        default=5,
        description="How many documents to return",
    )
    score_threshold: Optional[float] = Field(
        default=None,
        description="Optional similarity score cut-off (0-1).",
    )

# ---------- 2. The RAG tool ----------
class LlamaStackVectorStoreRAGTool(BaseTool):
    name: str = "Llama Stack Vector Store RAG tool"
    description: str = (
        "This tool calls a Llama-Stack endpoint for retrieval-augmented generation using a vector store. "
        "It takes a natural-language query and returns the most relevant documents."
    )
    args_schema: Type[BaseModel] = VectorStoreRAGToolInput
    client: Any
    vector_store_id: str = ""
    top_k: int = 5

    def _run(self, **kwargs: Any) -> str:
        # Helper function to extract actual values from CrewAI's Field metadata dicts
        def extract_value(val, default=None):
            """Extract actual value from CrewAI's Field metadata dictionary."""
            if isinstance(val, dict) and 'description' in val:
                # CrewAI passes Field metadata, extract the actual value
                # The dict looks like: {'description': 'actual_value', 'type': 'str'}
                return val.get('description', default)
            return val if val is not None else default
        
        # 1. Resolve parameters and extract actual values from CrewAI metadata
        query_raw = kwargs.get("query")
        vector_store_id_raw = kwargs.get("vector_store_id", self.vector_store_id)
        top_k_raw = kwargs.get("top_k", self.top_k)
        
        # Extract actual values
        query: str = extract_value(query_raw)
        vector_store_id: str = extract_value(vector_store_id_raw, self.vector_store_id)
        top_k: int = extract_value(top_k_raw, self.top_k)
        
        # Convert top_k to int if it's still a string
        if isinstance(top_k, str):
            top_k = int(top_k)
        
        if not vector_store_id or vector_store_id == "":
            print('vector_store_id is empty, please specify which vector_store to search')
            return "No documents found."
            
        # 2. Issue request to Llama-Stack
        response = self.client.vector_stores.search(
            vector_store_id=vector_store_id,
            query=query,
            max_num_results=top_k,
        )

        # 3. Massage results into a single human-readable string
        if not response or not response.data:
            return "No documents found."

        docs: List[str] = []
        for result in response.data:
            content = result.content[0].text if result.content else "No content"
            filename = result.filename if result.filename else {}
            docs.append(f"filename: {filename}, content: {content}")
        return "\\n".join(docs)

# Create tool instance with Llama Stack client
rag_tool = LlamaStackVectorStoreRAGTool(
    client=client,
    vector_store_id="",  # Will be provided at runtime
    top_k=3
)

print("\\n✅ RAG Tool configured successfully!")
print(f"   Tool Name: {rag_tool.name}")
print(f"   Uses: Llama Stack Vector Store API")

## Build the RAG Agent

### Create a Complete RAG Pipeline

Construct a CrewAI pipeline that orchestrates the RAG process:

1. **Agent Definition**: CrewAI agent with RAG capabilities
2. **Task Definition**: Task for answering questions using retrieved context
3. **Crew Definition**: Complete RAG pipeline

**CrewAI workflow**:
`User Query → CrewAI Task → Agent invokes RAG Tool → Llama Stack Vector Search → Retrieved Context → LLM Generation → Final Response`

In [ ]:
print("🤖 Creating CrewAI RAG agent...\\n")

# Define the RAG agent
agent = Agent(
    role="RAG assistant",
    goal="Answer user's questions with provided context",
    backstory="You are an experienced search assistant specializing in finding relevant information from documentation and vector_db to answer user questions accurately.",
    allow_delegation=False,
    llm=llm,
    tools=[rag_tool],
    verbose=True
)

# Define the task
task = Task(
    description="Answer the following question: {query}. Use the RAG tool to search the provided vector_store_id {vector_store_id} if needed.",
    expected_output="A clear and accurate answer to the question based on the retrieved context",
    agent=agent,
)

# Create the crew
crew = Crew(
    agents=[agent],
    tasks=[task],
    verbose=True
)

print("✅ RAG Pipeline created successfully!")
print("\\n📊 Configuration:")
print(f"   Agent Role: {agent.role}")
print(f"   Model: {INFERENCE_MODEL}")
print(f"   Tools: {len(agent.tools)} tool(s)")
print(f"   Framework: CrewAI")

## Testing the RAG System

### Example 1: Shipping Query

In [ ]:
print("\\n" + "="*60)
print("🚀 Testing CrewAI RAG Agent")
print("="*60 + "\\n")

query = "How long does shipping take?"
print(f"❓ {query}\\n")

response = crew.kickoff(inputs={"query": query, "vector_store_id": vector_store.id})

print("\\n" + "="*60)
print(f"💡 {response}")
print("="*60)

### Example 2: Returns Policy Query

In [ ]:
print("\\n" + "="*60)
print("🚀 Testing CrewAI RAG Agent")
print("="*60 + "\\n")

query = "What is the return policy?"
print(f"❓ {query}\\n")

response = crew.kickoff(inputs={"query": query, "vector_store_id": vector_store.id})

print("\\n" + "="*60)
print(f"💡 {response}")
print("="*60)

### Example 3: Support Query

In [ ]:
query = "How can I contact support?"
print(f"\\n❓ {query}\\n")

response = crew.kickoff(inputs={"query": query, "vector_store_id": vector_store.id})

print(f"\\n💡 {response}")

---

## Summary

We have successfully built a RAG system that combines:

- **Llama Stack** for LLM inference via OpenAI-compatible API
- **CrewAI** for agent orchestration (agents, tasks, and tools)
- **Custom RAG Tool** for document retrieval from knowledge base